In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

In [3]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
PATH = "data/road-damage/"
sz=250

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.backends.cudnn.enabled

True

In [7]:
os.listdir(PATH)

['train', 'test', 'valid']

In [8]:
os.listdir(f'{PATH}valid')

['Sumida', 'Nagakute', 'Chiba', 'Numazu', 'Muroran', 'Ichihara', 'Adachi']

WITHOUT TEST SET

In [9]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [10]:
data.classes

['Adachi', 'Chiba', 'Ichihara', 'Muroran', 'Nagakute', 'Numazu', 'Sumida']

In [11]:
data.val_y

array([0, 0, 0, ..., 6, 6, 6])

In [12]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(0.01, 2)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /content/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:01<00:00, 56468440.04it/s]


100%|██████████| 26/26 [00:21<00:00,  1.23it/s]


epoch      trn_loss   val_loss   accuracy                  
    0      0.910553   0.545993   0.807311  
    1      0.63099    0.435658   0.850062                  



[array([0.43566]), 0.8500619576470707]

FINDING LEARNING RATE

In [13]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [14]:
lrf=learn.lr_find()

 89%|████████▉ | 81/91 [00:01<00:00, 54.90it/s, loss=2.44]


In [21]:
learn.sched.plot_lr()

In [22]:
learn.sched.plot()

In [23]:
learn.fit(0.1, 5)

epoch      trn_loss   val_loss   accuracy                 
    0      1.046729   0.629405   0.782528  
    1      0.653579   0.419245   0.85316                   
    2      0.48262    0.367141   0.877943                  
    3      0.413183   0.360099   0.881041                  
    4      0.377566   0.364326   0.869888                  



[array([0.36433]), 0.8698884758364313]

In [24]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.2)

In [25]:
def get_augs():
    data = ImageClassifierData.from_paths(PATH, bs=16, tfms=tfms, num_workers=1)
    x,_ = next(iter(data.aug_dl))
    return data.trn_ds.denorm(x)[1]

In [26]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [27]:
learn.fit(0.1, 1)

epoch      trn_loss   val_loss   accuracy                 
    0      0.992499   0.541258   0.802354  



[array([0.54126]), 0.8023543990086741]

In [28]:
learn.precompute=False

In [29]:
learn.fit(0.1, 5, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                  
    0      0.781322   0.431922   0.839529  
    1      0.678897   0.408016   0.85254                   
    2      0.610464   0.409954   0.849442                  
    3      0.583694   0.392075   0.857497                  
    4      0.584593   0.370624   0.865551                  



[array([0.37062]), 0.8655514250309789]

In [30]:
learn.sched.plot_lr()

In [31]:
learn.unfreeze()

In [32]:
lr_rate = 0.1

In [33]:
lr=np.array([lr_rate/6,lr_rate/3,lr_rate])

In [34]:
learn.fit(lr, 5, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   accuracy                  
    0      0.262536   0.090252   0.967162  
    1      0.167647   0.059505   0.975217                  
    2      0.083912   0.033625   0.988228                   
    3      0.112605   0.051899   0.983271                   
    4      0.064608   0.039009   0.988228                   
    5      0.037469   0.021695   0.992565                   
    6      0.027052   0.023609   0.993185                   
    7      0.068525   0.071585   0.979554                   
    8      0.065494   0.062948   0.978315                   
    9      0.043569   0.027922   0.991326                   
    10     0.032542   0.014446   0.995663                   
    11     0.023031   0.026007   0.992565                   
    12     0.026449   0.018131   0.993185                   
    13     0.017667   0.02018    0.994424                   
    14     0.010389   0.020587   0.993804                   
    15     0.032329   0.025438   0.989467  

[array([0.02215]), 0.9950433705080545]

In [35]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [37]:
probs


array([[1.     , 0.     , 0.     , ..., 0.     , 0.     , 0.     ],
       [0.99999, 0.     , 0.     , ..., 0.     , 0.     , 0.     ],
       [1.     , 0.     , 0.     , ..., 0.     , 0.     , 0.     ],
       ...,
       [0.     , 0.     , 0.     , ..., 0.     , 0.     , 1.     ],
       [0.     , 0.     , 0.     , ..., 0.     , 0.     , 1.     ],
       [0.     , 0.00001, 0.     , ..., 0.     , 0.     , 0.99999]], dtype=float32)

In [38]:
accuracy_np(probs, y)

0.9950433705080545

In [39]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

In [40]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [41]:
plot_confusion_matrix(cm, data.classes)

[[291   0   0   0   0   0   0]
 [  0  81   0   0   0   0   0]
 [  0   0  45   0   0   0   0]
 [  0   2   0 376   1   0   0]
 [  1   0   0   0 251   2   1]
 [  0   0   0   0   1 356   0]
 [  0   0   0   0   0   0 206]]
